In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip3 install joblib==1.1.0
# !pip3 install openpyxl
# !pip3 install --user xlsxwriter
# !pip3 install ipython-autotime
# !pip3 install pickle5
# !pip3 install umap-learn
# !pip3 install hdbscan
import umap.umap_ as umap

%load_ext autotime
import pandas as pd
import numpy as np
import pickle5 as pickle
import re
# from nltk.stem import WordNetLemmatizer
# import nltk
# from textblob import TextBlob
from tqdm.notebook import tqdm
tqdm.pandas()
import requests
# from pandarallel import pandarallel

# import hdbscan
# import imagehash
import spacy
from collections import Counter
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import random
import itertools
import hdbscan
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import LogNorm
from matplotlib.lines import Line2D

nlp=spacy.load('en_core_web_sm')
# nb_workers=20
# pandarallel.initialize(nb_workers=nb_workers)
# my_dir=''
# my_dir='/content/drive/MyDrive/Meme_Polarity_Detection/'
# my_dir='/csg_vahid/'



def stratified_sample_df(df, col, n_samples):
    n = min(n_samples, df[col].value_counts().min())
    df_ = df.groupby(col).apply(lambda x: x.sample(n))
    df_.index = df_.index.droplevel(0)
    return df_
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

def vector_to_column(df,vectors,column_name):
    df[column_name]=0
    df[column_name]=df[column_name].astype(object)
    df.reset_index(drop=True,inplace=True)
    indices=df.index
    for i in indices:
        df.at[i, column_name]=vectors[i]
    return df

time: 11.1 s (started: 2022-11-29 12:36:15 +00:00)


# Pairwise Comparison of Two Communities

## Loading and Filtering the Commuinities Data

In [ ]:
my_dir='/content/drive/MyDrive/Discord-Reddit/'

time: 646 µs (started: 2022-11-29 12:36:32 +00:00)


In [ ]:
pd.read_hdf(my_dir + f"Datasets/vectorized_data/Overwatch_subreddit_hate.hdf")

,Unnamed: 0,comment_text,author,timestamp,hate,comment_text_cleaned,Toxic_BERT,all-mpnet-base-v2
0,140664,you've done a total of zero games. support suc...,ExpectFlames,2022-07-02 01:41:38,0.0,you've done a total of zero games. support suc...,0.697299,"[-0.025931455, -0.038999017, -0.03551868, -0.0..."
1,529574,make her the cow next,Apoptosis11,2022-03-17 07:35:36,0.0,make her the cow next,0.528157,"[0.04009053, 0.11188896, 0.010388376, -0.01440..."
2,515509,moria and throw piss balls around the map,jkappa11,2022-03-24 02:45:20,1.0,moria and throw piss balls around the map,0.846661,"[0.011725483, 0.002307051, -0.025434677, 0.020..."
3,671329,imagine eating corporate ass that much that vo...,JudiDenchsNeckVein,2022-01-18 17:12:40,0.0,imagine eating corporate ass that much that vo...,0.960193,"[-0.020192295, 0.11575708, 0.004626797, 0.0083..."
4,357229,"naw can't have that man, it would be clear tha...",Dreaming_Scholar,2022-05-01 17:41:30,0.0,"naw can't have that man, it would be clear tha...",0.937510,"[0.016178656, 0.06773032, -0.0085595, 0.064703..."
...,...,...,...,...,...,...,...,...
9995,300791,you are literally the only person that thinks ...,Eggs_McGee_nstuff,2022-05-14 20:02:44,1.0,you are literally the only person that thinks ...,0.944916,"[-0.053506747, 0.08655688, 0.04121974, 0.04143..."
9996,660527,enemy team dead. morons on your team already p...,Rdt_will_eat_itself,2022-01-21 16:24:26,0.0,enemy team dead. morons on your team already p...,0.993391,"[-0.054096505, -0.089433424, 0.010088789, 0.00..."
9997,360558,you know what sucks most? you can't go back to...,SerendipityLurking,2022-05-01 03:03:39,0.0,you know what sucks most? you can't go back to...,0.644750,"[-0.0014830527, 0.09363148, -0.030893989, -0.0..."
9998,693676,wtf 🤣,yourself88xbl,2022-01-09 23:16:20,0.0,wtf 🤣,0.849799,"[0.005985672, 0.07470488, 0.0064043636, 0.0014..."


time: 3.77 s (started: 2022-11-29 12:36:35 +00:00)


In [ ]:
def get_merged_data(file_name_x):
  discord_x = pd.read_hdf(my_dir + f"Datasets/vectorized_data/{file_name_x}_discord_hate.hdf")
  discord_x = discord_x[discord_x['content'].map(len)>5]
  print(f'{file_name_x}\' Discord has {len(discord_x)} data')
  reddit_x = pd.read_hdf(my_dir + f"Datasets/vectorized_data/{file_name_x}_subreddit_hate.hdf")
  reddit_x.rename({'comment_text':'content'},axis=1,inplace=True)
  reddit_x = reddit_x[reddit_x['content'].map(len)>5]
  print(f'{file_name_x}\' Reddit has {len(reddit_x)} data')
  discord_x['platform'] = 'discord'
  reddit_x['platform'] = 'reddit'
  smallest_size = min([len(discord_x),len(reddit_x)])

  discord_x = discord_x[['Unnamed: 0','content','all-mpnet-base-v2','platform']]
  reddit_x = reddit_x[['Unnamed: 0','content','all-mpnet-base-v2','platform']]
  discord_x = discord_x.sample(n=smallest_size,random_state=1)
  reddit_x = reddit_x.sample(n=smallest_size,random_state=1)
  merged_data_x = pd.concat([discord_x, reddit_x],ignore_index=True, axis=0)
  return merged_data_x

time: 4.92 ms (started: 2022-11-29 12:37:36 +00:00)


## Dimensionality Reduction to 2D

In [ ]:
def get_2D_plot_axes(data_x):
  manifold = umap.UMAP(n_components=2,
                    n_neighbors = 20,    
                    metric='cosine', verbose=True, 
                    random_state=1).fit(list(data_x['all-mpnet-base-v2']))
  embeddings_2D = manifold.transform(list(data_x['all-mpnet-base-v2']))
  data_x=vector_to_column(df = data_x, vectors=embeddings_2D, column_name='all-mpnet-base-v2_2D')
  x = [x for x, y in data_x['all-mpnet-base-v2_2D']]
  y = [y for x, y in data_x['all-mpnet-base-v2_2D']]
  return x,y

time: 1.82 ms (started: 2022-11-29 12:38:01 +00:00)


time: 3.03 s (started: 2022-08-25 17:17:49 +00:00)


## Plotting the Reduced Dimensions

In [ ]:
def plot_merged_data(x, y, data_x, community_name_x):

  graph_colors_dict = {'reddit':'red','discord':'blue'}

  sns.set_theme(font_scale=1.5, style = "white")
  plt.figure(figsize = (15,15),dpi=300) # Here's the part I need
  # plt.xlim(-5,15)
  # plt.ylim(-10, 15)
  plt.scatter(x, y, s=0.5, c = data_x["platform"].map(graph_colors_dict),marker="o")
  plt.title(f'2D projection of {community_name_x}',fontsize=25)
  # pop_r = mpatches.Circle(xy=(0.5,0.5),color=graph_colors_dict.get('TheRedPill'), label="TheRedPill",radius = 0.25)
  # pop_f = mpatches.Circle(xy=(0.5,0.5),color=graph_colors_dict.get('Feminism'), label='Feminism',radius = 0.25)
  pop_r = Line2D([0], [0],markerfacecolor=graph_colors_dict.get('reddit'), color='w', label='Reddit', marker="o", markersize=20)
  pop_f = Line2D([0], [0],markerfacecolor=graph_colors_dict.get('discord'), color='w', label='Discord', marker="o", markersize=20)
  # pop_58 = mpatches.Patch(color=graph_colors_dict.get(58), label='Forum 58')
  plt.legend(handles=[pop_r,pop_f], loc="upper right")
  plt.savefig(my_dir+f"Charts/Text_2D_Projection/{community_name_x}_2D_Projection", bbox_inches = 'tight',format="jpeg", dpi=550)
  plt.show()
  return None

time: 1.91 ms (started: 2022-11-29 12:38:40 +00:00)


time: 10.8 s (started: 2022-08-25 17:17:58 +00:00)


## Clustering Two Communities Together

### Step 1: Dimensionality Reduction to 10D using UMAP

### Step 2: Clustering the 10D version using HDBSCAN

In [ ]:
def generate_clusters(data_x,
                      n_neighbors,
                      n_components, 
                      min_cluster_size,
                      random_state):
    """
    Generate HDBSCAN cluster object after reducing embedding dimensionality with UMAP
    """
    
    umap_embeddings = (umap.UMAP(#n_neighbors=n_neighbors, 
                                n_components=n_components, 
                                metric='cosine', 
                                random_state=random_state, verbose=True)
                            .fit_transform(list(data_x['all-mpnet-base-v2'])))

    clusters = hdbscan.HDBSCAN(min_cluster_size = min_cluster_size,
                               metric='euclidean', 
                               cluster_selection_method='eom', core_dist_n_jobs=1
                               ).fit_predict(umap_embeddings)

    return clusters, umap_embeddings

time: 1.9 ms (started: 2022-11-29 12:38:44 +00:00)


In [ ]:
def get_cluster_proportion(data_x,base_column,true_value):
  data_x['proportion_bin'] = data_x[base_column].apply(lambda x: 1 if x==true_value else 0)
  proportion_dict = data_x.groupby('cluster')['proportion_bin'].mean().to_dict()
  # print(proportion_dict)
  return data_x['cluster'].map(proportion_dict)

time: 1.32 ms (started: 2022-11-29 12:38:48 +00:00)


time: 7.51 ms (started: 2022-08-25 16:58:54 +00:00)


## Generating a Label for each Cluster

In [ ]:
def extract_labels(category_docs, excluded_nouns):
    """
    Extract labels from documents in the same cluster by concatenating
    most common verbs, ojects, and nouns
    """

    verbs = []
    dobjs = []
    nouns = []
    adjs = []
    
    verb = ''
    dobj = ''
    noun1 = ''
    noun2 = ''

    # for each document, append verbs, dobs, nouns, and adjectives to 
    # running lists for whole cluster
    for i in range(len(category_docs)):
        doc = nlp(category_docs[i])
        for token in doc:
            if token.is_stop==False:
                if token.dep_ == 'ROOT':
                    verbs.append(token.text.lower())

                elif token.dep_=='dobj':
                    dobjs.append(token.text.lower())

                elif token.pos_=='NOUN':
                    nouns.append(token.text.lower())
                    
                elif token.pos_=='ADJ':
                    adjs.append(token.text.lower())

    verbs = Counter(verbs)
    dobjs = Counter(dobjs)
    nouns = Counter(nouns)
    
    for word in list(verbs):
        if word in excluded_nouns:
            del verbs[word]
    del verbs['think']
            
    for word in list(dobjs):
        if word in excluded_nouns:
            del dobjs[word]
    
    for word in list(nouns):
        if word in excluded_nouns:
            del nouns[word]

    if len(verbs) > 0:
        verb = verbs.most_common(1)[0][0]

    if len(dobjs) > 0:
        dobj = dobjs.most_common(1)[0][0]

    if len(nouns) > 0:
        noun1 = nouns.most_common(1)[0][0]

    if len(set(nouns)) > 1:
        noun2 = nouns.most_common(2)[1][0]
    
    # concatenate the most common verb-dobj-noun1-noun2 (if they exist)
    label_words = [verb, dobj]
    
    for word in [noun1, noun2]:
        if word not in label_words:
            label_words.append(word)
    
    if '' in label_words:
        label_words.remove('')
    
    label = '_'.join(label_words)
    
    return label

time: 7 ms (started: 2022-11-29 12:38:52 +00:00)


In [ ]:
def text_cluster_to_label(data_x,excluded_nouns):
      clusters_dict = dict()
      clusters_set = set(data_x['cluster'])
      for cluster_i in clusters_set:
                label_ = extract_labels(list(data_x[data_x['cluster']==cluster_i]['content']),excluded_nouns=excluded_nouns)
                clusters_dict.update({cluster_i:label_})
      return data_x['cluster'].map(clusters_dict), clusters_dict

time: 1.15 ms (started: 2022-11-29 12:38:57 +00:00)


# Main

In [ ]:
my_dir = '/content/drive/MyDrive/Discord-Reddit/'

community_names = ['Overwatch','europe','Jokes','nosleep','Rainbow6','history','Ukrainian','ksi','sports',
 'dankmemes','Games','kpop','music','rickandmorty','Writingprompts']
# community_names = ['dankmemes','Games','kpop','music','rickandmorty','Writingprompts']
# community_names = ['Ukrainian','Overwatch','europe','Rainbow6','ksi',
#  'dankmemes','Games','kpop','music','rickandmorty','Writingprompts']

time: 1.13 ms (started: 2022-11-29 12:43:30 +00:00)


In [ ]:
for community_name in community_names:
  print(f'beginning {community_name}')
  merged_data = get_merged_data(file_name_x=community_name)
  x , y = get_2D_plot_axes(data_x = merged_data)
  plot_merged_data(x=x, y=y, data_x=merged_data, community_name_x = community_name)
  merged_data['cluster'], embeddings_10D = generate_clusters(data_x = merged_data,
                                  n_neighbors = 20,
                                  n_components= 10, 
                                  min_cluster_size = 10,
                                  random_state = 1)
  merged_data=vector_to_column(df = merged_data, vectors = embeddings_10D, column_name = 'all-mpnet-base-v2_10D')
  merged_data['proportion_of_reddit'] = get_cluster_proportion(data_x=merged_data,base_column='platform', true_value = 'reddit')
  merged_data['cluster_label'],clusters_dict=text_cluster_to_label(merged_data,excluded_nouns=[])
  merged_data[['Unnamed: 0','content','platform','cluster','cluster_label','proportion_of_reddit']].to_excel(my_dir+f'Datasets/Clustered_Data/{community_name}_clustered.xlsx',index=None)

Output hidden; open in https://colab.research.google.com to view.